# Behavioural cloning project

In [1]:
# Configure matlab to show graphics in the notebook
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# Change to 'data' when training on a GPU
PATH_TO_DATA = 'all_data'

In [3]:
# Shape of the image
input_shape = (66, 200, 3)

In [4]:
def img_folder(data_folder):
    return '{}/IMG'.format(data_folder)

def path_driving_log(data_folder):
    return '{}/driving_log.csv'.format(data_folder)

In [5]:
datasets = ['train', 'test', 'valid']

In [6]:
def get_driving_log_dataframe(data_folder):
    driving_log_df = pd.read_csv(path_driving_log(data_folder))
    return driving_log_df

In [7]:
path_to_folders = dict(zip(datasets, map(lambda folder: '{0}/{1}'.format(PATH_TO_DATA, folder), datasets)))

In [8]:
path_to_folders

{'test': 'all_data/test', 'train': 'all_data/train', 'valid': 'all_data/valid'}

In [9]:
!pip install imutils

  Running setup.py bdist_wheel for imutils ... - \ done
  Stored in directory: /home/carnd/.cache/pip/wheels/4c/a7/0f/2ac8bb8d2d9f472f26413454a7ae30d96cd9f645c9f526a2b9
Successfully built imutils


In [10]:
import imutils

In [75]:
from scipy.ndimage import imread
from os import listdir
from sklearn.utils import shuffle

from PIL import Image

def image_label_generator(data_folder, batch_size=64):
    driving_log_df = get_driving_log_dataframe(data_folder)
    number_of_examples = len(driving_log_df)
    image_columns = ['center', 'left', 'right']
    
    X_train = []
    y_train = []
    weights = []
    index_in_batch = 0
    batch_number = 0
    
    while True:
        for image_column in image_columns:
            image_series = driving_log_df[image_column]
            steering_series = driving_log_df['steering']
            for offset in range(0, number_of_examples, batch_size):
                X_train = []
                y_train = []
                weights = []

                end_of_batch = min(number_of_examples, offset + batch_size)

                for j in range(offset, end_of_batch):
                    image_filename = image_series[j].lstrip().rstrip()
                    image = Image.open('{0}/{1}'.format(data_folder, image_filename))
                    image = np.asarray(image.resize((200, 66)))
                    label = steering_series[j]
                    
#                     weights.append(1)
                    if abs(label) < 10e-4:
                        weights.append(1)
                    else:
                        if label > 0:
                            weights.append(5)
                        else:
                            weights.append(3)
                    
                    
                    X_train.append(image)
                    y_train.append(label)
                    X_train, y_train, weights = shuffle(X_train, y_train, weights)

                yield np.array(X_train), np.array(y_train), np.array(weights)

In [76]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import Flatten
from keras.layers import Dropout

In [77]:
model = Sequential()

In [78]:
model.add(Convolution2D(3, 1, 1, 
                          input_shape=input_shape, 
                          border_mode='same', 
                          activation='relu',
                          init='he_normal'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Convolution2D(24, 
                        5, 5,
                       subsample=(2, 2),
                       init='he_normal'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Convolution2D(36, 
                        5, 5,
                       subsample=(2, 2),
                       init='he_normal'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Convolution2D(48, 
                        5, 5,
                       subsample=(2, 2),
                       init='he_normal'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Convolution2D(64,
                       3, 3,
                       init='he_normal'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Convolution2D(64,
                       3, 3,
                       init='he_normal'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100, activation='relu', init='he_normal'))
model.add(Dense(50, activation='relu', init='he_normal'))
model.add(Dense(10, activation='relu', init='he_normal'))
model.add(Dense(1, activation='linear', init='he_normal'))

In [79]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_31 (Convolution2D) (None, 66, 200, 3)    12          convolution2d_input_6[0][0]      
____________________________________________________________________________________________________
dropout_14 (Dropout)             (None, 66, 200, 3)    0           convolution2d_31[0][0]           
____________________________________________________________________________________________________
batchnormalization_22 (BatchNorm (None, 66, 200, 3)    12          dropout_14[0][0]                 
____________________________________________________________________________________________________
convolution2d_32 (Convolution2D) (None, 31, 98, 24)    1824        batchnormalization_22[0][0]      
___________________________________________________________________________________________

In [80]:
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath)
callbacks_list = [checkpoint]

In [81]:
from keras.optimizers import Adam
adam = Adam(lr=10e-4)

In [82]:
model.compile(optimizer=adam, 
              loss='mse',
             metrics=['mean_squared_error'])

In [83]:
image_generator = image_label_generator(PATH_TO_DATA)
samples = len(get_driving_log_dataframe(PATH_TO_DATA))

In [84]:
model.fit_generator(image_generator, 
                    samples_per_epoch=samples, 
                    nb_epoch=18,
                   callbacks=callbacks_list)

Epoch 1/18
14816/14816 [==============================] - 51s - loss: 0.1689 - mean_squared_error: 0.0575    
Epoch 2/18
14816/14816 [==============================] - 48s - loss: 0.1218 - mean_squared_error: 0.0346    
Epoch 3/18
14816/14816 [==============================] - 48s - loss: 0.1153 - mean_squared_error: 0.0312    
Epoch 4/18
14816/14816 [==============================] - 48s - loss: 0.1124 - mean_squared_error: 0.0302    
Epoch 5/18
14816/14816 [==============================] - 48s - loss: 0.1098 - mean_squared_error: 0.0303    
Epoch 6/18
14816/14816 [==============================] - 48s - loss: 0.0981 - mean_squared_error: 0.0277    
Epoch 7/18
14816/14816 [==============================] - 48s - loss: 0.0902 - mean_squared_error: 0.0264    
Epoch 8/18
14816/14816 [==============================] - 48s - loss: 0.0829 - mean_squared_error: 0.0255    
Epoch 9/18
14816/14816 [==============================] - 48s - loss: 0.0780 - mean_squared_error: 0.0249    
Epoch 10/1

In [85]:
X_val, y_val, w = next(image_label_generator(PATH_TO_DATA))

In [86]:
model.predict(X_val)

array([[-0.08967546],
       [-0.06616151],
       [-0.10278946],
       [-0.09259585],
       [-0.10940063],
       [-0.10381505],
       [-0.06627685],
       [ 0.18588325],
       [-0.08025751],
       [-0.10272971],
       [-0.08930337],
       [-0.11079952],
       [-0.15881106],
       [-0.13396287],
       [-0.0843271 ],
       [-0.07536602],
       [-0.19298965],
       [-0.03857133],
       [-0.1059418 ],
       [-0.11409095],
       [-0.10841718],
       [ 0.18588325],
       [-0.14776227],
       [-0.11993295],
       [-0.17805156],
       [ 0.18588325],
       [-0.07372436],
       [-0.08802262],
       [-0.06658542],
       [-0.18182668],
       [-0.11287931],
       [-0.14715546],
       [-0.09928486],
       [-0.06285028],
       [-0.15824306],
       [-0.15553769],
       [-0.14736149],
       [ 0.18588325],
       [-0.18080536],
       [-0.09366712],
       [-0.09812418],
       [-0.11197266],
       [-0.11008781],
       [-0.16207066],
       [-0.09614736],
       [ 0

In [87]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [31]:
y_val

array([-0.04076847, -0.1167233 ,  0.07132844,  0.        ,  0.07132844,
       -0.05975719,  0.        ,  0.        ,  0.1670138 , -0.09773462,
       -0.05975719,  0.0617599 ,  0.0617599 ,  0.1670138 ,  0.1670138 ,
        0.01391724,  0.        ,  0.0617599 ,  0.07132844])